In [1]:
import girder_client as gc
import json
import collections
import pymongo    
import dateutil.parser
import datetime as dt
BASE_URL = 'https://isic-archive.com/girder/api/v1'
gc = gc.GirderClient(apiUrl=BASE_URL)

In [2]:
### I hard coded the LesionImageFolder ID--- this is where all the Annotaiton Data lives
AnnotationStudiesCollectionId = '55943cff9fc3c13155bcad61'

# LesionImageFolderId='573f11119fc3c132505c0ee7'  #<<<This is the FIRST STUDY ACTUALLY
# LIFdata = gc.listFolder(LesionImageFolderId)

AnnotationStudies = gc.listFolder(AnnotationStudiesCollectionId,parentFolderType='Collection')

In [3]:
### Get folderInfo for a rater.... this stores what images they marked up
## I want to have a nested dictionary as follows   STUDY_NAME --> IMAGE_NAME --> ANNOTATORS
ReformmedStudyDict = {}


fieldsToRemove =  ['size','creatorId','baseParentId','license','description','_modelType','folderId', 'baseParentType','created', '_id']


metaFieldsToRemove = ['studyId','userId','imageId']


for cs in AnnotationStudies:
    CurStudyName = cs['name']
    print CurStudyName,"is being Analyzed"
    if CurStudyName not in ReformmedStudyDict:
        ReformmedStudyDict[CurStudyName] = {}
        ReformmedStudyDict[CurStudyName]['MarkupData'] = {}


    ## Now get the data for the current Study
    LIFdata = gc.listFolder(cs['_id'])
    
    
    if 'FeatureSetId' not in ReformmedStudyDict[CurStudyName]:
        ReformmedStudyDict[CurStudyName]['FeatureSetId'] = cs['meta']['featuresetId']
    for LIF in LIFdata:
        curUserName = LIF['name'].split()[0]
        print curUserName,"is being processed"
        curUserAnnotationList = gc.listItem( LIF['_id'])

        for cua in curUserAnnotationList:
            imgName =  cua['name']
            annotationInfo = cua['meta']
            ### I'm going to strip out keys I don't need?
            ### Need to see if this Image has been seen
            if imgName not in ReformmedStudyDict[CurStudyName]['MarkupData']:
                ReformmedStudyDict[CurStudyName]['MarkupData'][imgName] = {}
                ReformmedStudyDict[CurStudyName]['MarkupData'][imgName]['raters'] = {}
           
            try:
                ## See if current annotation has a status of ok
                curStatus = cua['meta']['status']
                #print curStatus
                ## if this key is not there, means the user never completed the nanotations
                
            except:
                break
                
            if (curUserName not in ReformmedStudyDict[CurStudyName]['MarkupData'][imgName]['raters']):
                cuaClean = cua.copy()
                
                for ftr in fieldsToRemove:
                    if ftr in cuaClean:
                        del cuaClean[ftr]

                for ftr in metaFieldsToRemove:
                    if ftr in cuaClean['meta']:
                        del cuaClean['meta'][ftr]
                ### now clean up the metadata object as well.. too much data is being returend
                stop=start=None
                #meta fields to delete
                if 'stopTime' in cuaClean['meta']:
                    stop = dateutil.parser.parse(cuaClean['meta']['stopTime'])
                    del cuaClean['meta']['stopTime']
                if 'startTime' in cuaClean['meta']:
                    start = dateutil.parser.parse(cuaClean['meta']['startTime'])
                if stop and start:

                    cuaClean['meta']['markupTime'] = "%s" % (stop-start)
                
                
                ReformmedStudyDict[CurStudyName]['MarkupData'][imgName]['raters'][curUserName] = cuaClean

                
with open('momo.json','w') as fp:
    json.dump(ReformmedStudyDict,fp,indent=1)

ISBI 2016: 100 Lesion Classification is being Analyzed
carrerac is being processed
halperna is being processed
jldefazio is being processed
marghooa is being processed
njaimes is being processed
oyelamos is being processed
quigleye is being processed
scopea1 is being processed
UDA-1 Testing is being Analyzed
aadikalloo is being processed
brianhelba is being processed
davidmanthey is being processed
konstantinosliopyris is being processed
msmolens is being processed


In [6]:
AnnotationStudies

[{u'_accessLevel': 0,
  u'_id': u'573f11119fc3c132505c0ee7',
  u'_modelType': u'folder',
  u'baseParentId': u'55943cff9fc3c13155bcad61',
  u'baseParentType': u'collection',
  u'created': u'2016-05-20T13:28:49.992000+00:00',
  u'creatorId': u'54cb967ebae47819d8e4c724',
  u'description': u'',
  u'meta': {u'featuresetId': u'573f09bd9fc3c132505c0ee6'},
  u'name': u'ISBI 2016: 100 Lesion Classification',
  u'parentCollection': u'collection',
  u'parentId': u'55943cff9fc3c13155bcad61',
  u'public': True,
  u'size': 0,
  u'updated': u'2016-11-08T10:25:45.485000+00:00'},
 {u'_accessLevel': 0,
  u'_id': u'57aa26699fc3c1668bb7f9e3',
  u'_modelType': u'folder',
  u'baseParentId': u'55943cff9fc3c13155bcad61',
  u'baseParentType': u'collection',
  u'created': u'2016-08-09T18:52:25.589000+00:00',
  u'creatorId': u'55d4cde89fc3c1490e995086',
  u'description': u'',
  u'meta': {u'featuresetId': u'55c3d24c9fc3c149ba6470e1'},
  u'name': u'UDA-1 Testing',
  u'parentCollection': u'collection',
  u'parentId

In [10]:
#!pip install python-dateutil==2.4.2

In [13]:
# print ReformmedStudyDict.keys()
# print ReformmedStudyDict['ISBI Lesion Classification'].keys()

for studyName,studyAnnotationData in ReformmedStudyDict.iteritems():
    print "------%s----" % studyName
    for img in studyAnnotationData['MarkupData']:
        print img, len(studyAnnotationData['MarkupData'][img]['raters'])

with open('momo.json','w') as fp:
    json.dump(ReformmedStudyDict,fp,indent=2)
        

 ------UDA-1 Testing----
ISIC_0000169 2
ISIC_0000297 1
ISIC_0000316 1
ISIC_0000160 2
ISIC_0000166 2
ISIC_0000315 1
ISIC_0000313 1
ISIC_0000262 1
ISIC_0000295 1
ISIC_0000199 1
ISIC_0000235 1
ISIC_0000319 1
ISIC_0000296 1
ISIC_0000210 1
ISIC_0000314 1
ISIC_0000212 1
ISIC_0000190 1
ISIC_0000311 1
ISIC_0000310 1
ISIC_0000238 1
ISIC_0000312 1
ISIC_0000273 1
ISIC_0000252 1
ISIC_0000253 1
ISIC_0000306 1
ISIC_0000304 1
ISIC_0000170 1
ISIC_0000173 1
ISIC_0000158 3
ISIC_0000298 1
ISIC_0000299 1
ISIC_0000178 1
ISIC_0000213 1
ISIC_0000198 1
ISIC_0000318 1
ISIC_0000320 1
ISIC_0000209 1
ISIC_0000269 1
ISIC_0000187 1
ISIC_0000302 1
ISIC_0000303 1
ISIC_0000300 1
ISIC_0000301 1
ISIC_0000243 1
ISIC_0000307 1
ISIC_0000241 1
ISIC_0000305 1
ISIC_0000308 1
ISIC_0000309 1
ISIC_0000317 1
------ISBI 2016: 100 Lesion Classification----
ISIC_0000022 8
ISIC_0000287 8
ISIC_0000040 8
ISIC_0000510 8
ISIC_0000311 8
ISIC_0000289 8
ISIC_0000141 8
ISIC_0000164 8
ISIC_0000518 8
ISIC_0000499 8
ISIC_0009955 8
ISIC_0009958 

In [ ]:
print len(LIFdata) ## These are the 8 raters.. what I care about is the data from each rater..

LIFdataClean = []
for lf in  LIFdata:
    #print lf.keys()
#    [u'size', u'updated', u'description', u'created', u'baseParentType', u'public', u'meta', u'baseParentId', u'creatorId', u'parentId', u'parentCollection', u'_id', u'_modelType', u'_accessLevel', u'name']
    keysToKeep = [  u'meta', u'baseParentId',  u'parentId', u'_id',u'name']
    
    lfClean = lf.copy()
    for k,v in lf.iteritems():
        if k not in keysToKeep:
            del lfClean[k]
    LIFdataClean.append(lfClean)
    
#     for k,v in lf.iteritems():
#         if k in keysToKeep:
#             print k,v
#     sys.exit()
print LIFdataClean

In [ ]:
from pprint import pprint

In [ ]:
### The data is currently nearly impossible to decipher based on how it's stored
### I want to basically know the Study Names, the images in that study, the people who rated it, and then their answers...

### First look in the Annotation Studies Folder
AnnotStudyColl = '55943cff9fc3c13155bcad61'
AnnotStudies = gc.listFolder(AnnotStudyColl,parentFolderType='collection')



In [ ]:
AnnotStudiesClean = []
for ast in AnnotStudies:
    AnnotStudiesClean.append( {'_id': ast['_id'], 'studyId': ast['_id'], 'studyName': ast['name'], 
                             'studyFeatureSetId': ast['meta']['featuresetId']} )

In [ ]:
pprint( AnnotStudiesClean)

featureSetList = gc.getResource('featureset')


In [ ]:
featureSetData = {}
for fsl in featureSetList:
    curFeatSet = gc.getResource('featureset/%s' % fsl['_id'])
#     print curFeatSet
#     print fsl['name']
    featureSetData[fsl['name']] = curFeatSet

In [ ]:
print featureSetData.keys()

In [ ]:
pmc = pymongo.MongoClient('localhost',27017)
pmc.database_names()
ISIC_DB = pmc['ISIC_DB']

In [ ]:
https://isic-archive.com/girder#collection/55943cff9fc3c13155bcad61/folder/573f11119fc3c132505c0ee7